In [1]:
from sys import path
path.append("..")

import monte_carlo_spin_model
from   monte_carlo_spin_model.spin_system         import TwoDimensionalSpinLattice
from   monte_carlo_spin_model.spin_model          import IsingModel
from   monte_carlo_spin_model.annealing_algrithm  import equilibrate, annealing

from numpy import tanh, cosh

import matplotlib as mpl
from   matplotlib import pyplot as plt

In /Users/yangjunjie/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/yangjunjie/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/yangjunjie/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/yangjunjie/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/yangjunjie/opt

In [2]:
coulping_const = 1.0
nn             = 200

ising_model = IsingModel(interaction_strength=coulping_const)
spin_lattice  = TwoDimensionalSpinLattice(ising_model, shape=nn, is_pbc=True)

for i in range(spin_lattice.size):
    site = spin_lattice.get_site(i)

In [3]:
site1          = spin_lattice.get_site(1)
site1_config   = spin_lattice.get_index_config(1)

site2         = spin_lattice.get_site(2)
prev_config   = site2.get_config()

site3          = spin_lattice.get_site(3)
site3_config   = spin_lattice.get_index_config(3)

print("site1  config      = {: d}".format(site1_config))
print("site2  prev_config = {: d}".format(prev_config))
print("site3  config      = {: d}".format(site3_config))

site1  config      =  1
site2  prev_config = -1
site3  config      = -1


In [4]:
prev_config   = site2.get_config()
print("site2  prev_config    = {: d}".format(prev_config))

spin_lattice.sweep_site(site2)
current_config = site2.get_config()
print("site2  current_config = {: d}".format(current_config))

site2  prev_config    = -1


TypeError: 'NoneType' object is not iterable

In [ ]:
spin_lattice.sweep_system()
print("spin_lattice.get_system_energy() = %f"%(spin_lattice.get_system_energy()))
for i in range(spin_lattice.size):
    site = spin_lattice.get_site(i)
#     print("site id = {:d}, config = {: d}".format(id(site), site.get_config()))

In [ ]:
equilibrate(spin_lattice, temperature=10.0, max_cycle=2000, tol=1e-4)

In [ ]:
temp_list, ene_list, cv_list = annealing(
    spin_lattice, temp_init=5.0, temp_final=0.1, nsteps=20, max_cycle=2000, tol=1e-4
)

In [ ]:
style_dict = {
    'Metropolis algorithm': dict(linestyle='-.', marker='s', color='C1',  markersize=6, markevery=2),
    'Exact calculation':    dict(linestyle='-',  marker='*', color='C2',  markersize=6, markevery=2),
}

params = {
        "axes.titlesize":  10, 
        "axes.labelsize":  20, 
        "legend.fontsize": 15,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10, 
        "xtick.direction":'in', 
        "ytick.direction":'in' 
}
mpl.rcParams.update(**params)

exact_ene_list = [-nn*tanh(coulping_const/t)                       for t in temp_list]
exact_cv_list  = [ nn*coulping_const/cosh(coulping_const/t)**2/t/t for t in temp_list]

fig1, ax1 = plt.subplots(2,1,figsize=(10,8),sharex=True)
fig1.subplots_adjust(left=0.09, bottom=0.1, right=0.99, top=0.99, wspace=0.1)
ax1[0].plot(temp_list, ene_list,       **style_dict['Metropolis algorithm'], label='Metropolis algorithm')
ax1[0].plot(temp_list, exact_ene_list, **style_dict['Exact calculation'],    label='Exact calculation')

ax1[1].plot(temp_list, cv_list,        **style_dict['Metropolis algorithm'], label='Metropolis algorithm')
ax1[1].plot(temp_list, exact_cv_list,  **style_dict['Exact calculation'],    label='Exact calculation')

props = {'ylabel':r'$\langle E \rangle$'}
ax1[0].grid(linestyle="--", alpha=0.2)
ax1[0].set(**props)

props = {'ylabel':r'$\langle C_v \rangle$', 'xlabel':r'$T$'}
ax1[1].legend(loc='upper right')
ax1[1].grid(linestyle="--", alpha=0.2)
ax1[1].set(**props);